In [1]:
import os

# Set the proxy environment variables
os.environ['http_proxy'] = 'http://127.0.0.1:7890'
os.environ['https_proxy'] = 'http://127.0.0.1:7890'

In [2]:
import ee
import datetime
import json
import os
import itertools
import sys
import re
from glob import glob
from tqdm.auto import tqdm
from dfply import *

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import geemap

import subprocess
from subprocess import PIPE
import ast

In [3]:
ee.Initialize()

In [4]:
# append upper folder into sys-path during run time so we can
# import our own moduls
sys.path.append('../../')

In [5]:
from BackGround_modules.Class_2_Classify_Fourier_Img   import Classification
from BackGround_modules.Class_3_Calculate_the_accuracy import Accuracy_assesment

##### Prepare basic parameters
| Image    | Sample          | Classification |
|:---------|:---------------|:--------------|
| dongbei  | dongbei         | dongbei        |
| huabei   | huabei          | huabei         |
| huadong  | ★zhongnan       | huadong        |
| xibei    | ★xibei&★zhongnan| xibei          |
| xinan    | ★huadong       | xinan          |
| zhongnan | zhongnan        | zhongnan       |


In [6]:
# define the region to analysis
region_cns = ['华东',   '东北',   '中南',    '华北',  '西北', '西南']
region_ens = ['huadong','dongbei','zhongnan','huabei','xibei','xinan']

In [11]:
# define the region in this single run
region_cn = '中南'
region_en = 'zhongnan'

# do you want to use the sample pts from other region?
other_sample = 'zhongnan'
# other_sample = region_en

In [22]:
# define the years to be classified
year_name  = [f'{i}_{i+2}' for i in range(1990,2023,3)]

# define the number of total periods
num_invarient = len(year_name)

In [23]:
year_name

['1990_1992',
 '1993_1995',
 '1996_1998',
 '1999_2001',
 '2002_2004',
 '2005_2007',
 '2008_2010',
 '2011_2013',
 '2014_2016',
 '2017_2019',
 '2020_2022']

In [132]:
# import China shp boundary
Region =  ee.FeatureCollection("users/wangjinzhulala/China_built_up/01_Boundary_shp/classification_boundary_patch")

# Define the path to Sample_pt
sample_img_path = f'users/wangjinzhulala/China_built_up/04_sample_train_test_split'
export_path = 'users/wangjinzhulala/China_built_up/06_temporal_corrected_classification'

##### Fetch all imgs

In [114]:
# read all asset_df into a list
asset_list = [pd.read_csv(f) for f in glob('../../File_trainsition/gee_asset_china/*.csv')]

# concate all asset_df into one df
asset_df = pd.concat(asset_list)
asset_df.sort_values(['region','type','year'],inplace=True)

# mosaic all imgs of different regions into a single one
asset_df = asset_df.groupby(['year','type'])['path']\
        .apply(lambda imgs:ee.ImageCollection(imgs.tolist()).mosaic()).reset_index()

##### Prepare input_image for classification

Some additional treatment for the last two periods of Landsat and Sentinel imgs

In [117]:
# import Landsat img
Landsat_df = asset_df[(asset_df['type']=='Landsat_cloud_free')]
Landsat_img = [ee.Image(i) for i in  Landsat_df['path'].values]

# import Fourier img   
Fourier_df  = asset_df[(asset_df['type']=='Fourier')]
Fourier_img = [ee.Image(i) for i in  Fourier_df['path'].values]

# import Normalized img   
Normalized_df  = asset_df[(asset_df['type']=='NDVI_NDBI_EVI')]
Normalized_img = [ee.Image(i) for i in  Normalized_df['path'].values]

# Import DEM/SLOPE Img
DEM   = [ee.Image("USGS/SRTMGL1_003").rename('DEM')] * num_invarient
SLOPE = [ee.Terrain.slope(DEM).rename('SLOPE')] * num_invarient

In [118]:
# stack all Imput_Img together
Stack_img = [ee.Image(img) for img in zip(Landsat_img,
                                           Fourier_img,
                                           Normalized_img,
                                           DEM,
                                           SLOPE)]

##### Prepare input sample points

In [124]:
# remove climate properties
select_ppts = ['B1', 'B10', 'B11', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'Built', 'DEM', 'EVI', 
               'EVI_constant', 'EVI_cos_1', 'EVI_cos_2', 'EVI_cos_3', 'EVI_sin_1', 'EVI_sin_2', 'EVI_sin_3',
               'EVI_t', 'NDBI', 'NDBI_constant', 'NDBI_cos_1', 'NDBI_cos_2', 'NDBI_cos_3', 'NDBI_sin_1', 
               'NDBI_sin_2', 'NDBI_sin_3', 'NDBI_t', 'NDVI', 'NDVI_constant', 'NDVI_cos_1',
               'NDVI_cos_2', 'NDVI_cos_3', 'NDVI_sin_1', 'NDVI_sin_2', 'NDVI_sin_3', 'NDVI_t', 'SLOPE']

remove_ppts = ['lrad','prec','pres','shum', 'srad', 'temp','wind']

# Get the training sample
Training_sample = [ee.FeatureCollection(f"{sample_img_path}/Training_sample_{other_sample}_{year}")
                   for year in year_name]

Training_sample = [fe_col.map(lambda fe:fe.select(select_ppts))
                   for fe_col in Training_sample]

##### Stack input_img and input_sample together

In [125]:
# first put all inputs together
stack_year   = year_name
stack_img    = Stack_img 
stack_sample = Training_sample 

# stack all ingredients together
Stack_img_sample = list(zip(stack_year,stack_img,stack_sample))

#### Create the classification instances

In [126]:
# import the inbands-combination 
in_band_df_before_2020 = pd.read_csv('../Sub_Process_7_Before_classification_Sensitivity_test/Result/In_bands_combination.csv')
in_band_2017_2019  = in_band_df_before_2020 >> mask(X.year_range=='2017_2019')

if num_invarient > 10:
    in_band_df = pd.concat([in_band_df_before_2020]+[in_band_2017_2019]*(num_invarient-10),0)
    in_band_df['year_range'] = year_name

# selections for in-band combination
in_band_selection =[#'Spectrum',
                    #'Spectrum_Normalize',
                    #'Spectrum_Normalize_Fourier',
                    #'Spectrum_Normalize_Fourier_Terrain',
                    'Spectrum_Normalize_Fourier_Terrain_Meterology']

In [127]:
# A three-layer iteration to loopthrough [inbands/year/10-randome-split] individualy
Classificatioin_result = []


# remove the targe property
if 'Built' in in_features:
    in_features.remove('Built')

# The first-layer loop to go througth each inbands
for input_bands in in_band_selection:

    # The second-layer loop to go througth each year-periods
    for i,input_variable in enumerate(Stack_img_sample):
        
        
        # Fetch basic parameters
        classificaiton_tpye = input_bands
        year                = input_variable[0]
        input_img           = input_variable[1]
        sample_pt           = input_variable[2]

        # here determine which bands are involved in the classification
        # because the list is a string represented format, here use ast 
        # module to conver it back to a Python list
        in_features  =  ast.literal_eval(in_band_df.iloc[i][input_bands])
        for item in remove_ppts:
            in_features.remove(item)

            
        # start classification 
        classifier = ee.Classifier.smileRandomForest(100)
        trained_classifier = classifier.train(sample_pt, 'Built', in_features)
        classified_img = input_img.classify(trained_classifier).clip(Region)


        #__________________________________Add all classification result into a list____________________________
        Classificatioin_result.append((classificaiton_tpye,year,classified_img))

In [128]:
# convert classificaiton instances into a dataframe
Classification_instances_df =  pd.DataFrame(Classificatioin_result,
                                            columns=['classificaiton_tpye','year','classified_img'])
Classification_instances_df

,classificaiton_tpye,year,classified_img
0,Spectrum_Normalize_Fourier_Terrain_Meterology,1990_1992,"ee.Image({\n ""functionInvocationValue"": {\n ..."
1,Spectrum_Normalize_Fourier_Terrain_Meterology,1993_1995,"ee.Image({\n ""functionInvocationValue"": {\n ..."
2,Spectrum_Normalize_Fourier_Terrain_Meterology,1996_1998,"ee.Image({\n ""functionInvocationValue"": {\n ..."
3,Spectrum_Normalize_Fourier_Terrain_Meterology,1999_2001,"ee.Image({\n ""functionInvocationValue"": {\n ..."
4,Spectrum_Normalize_Fourier_Terrain_Meterology,2002_2004,"ee.Image({\n ""functionInvocationValue"": {\n ..."
5,Spectrum_Normalize_Fourier_Terrain_Meterology,2005_2007,"ee.Image({\n ""functionInvocationValue"": {\n ..."
6,Spectrum_Normalize_Fourier_Terrain_Meterology,2008_2010,"ee.Image({\n ""functionInvocationValue"": {\n ..."
7,Spectrum_Normalize_Fourier_Terrain_Meterology,2011_2013,"ee.Image({\n ""functionInvocationValue"": {\n ..."
8,Spectrum_Normalize_Fourier_Terrain_Meterology,2014_2016,"ee.Image({\n ""functionInvocationValue"": {\n ..."
9,Spectrum_Normalize_Fourier_Terrain_Meterology,2017_2019,"ee.Image({\n ""functionInvocationValue"": {\n ..."


#### Add one of the classified img to map

In [129]:
# define parameters of classification for show in map
bands_in_type = in_band_selection[0]
year_in       = '2020_2022'


# get the classification img
one = (Classification_instances_df >> mask(X.classificaiton_tpye == bands_in_type,
                                  X.year == year_in))['classified_img'].values[0]

In [130]:
# add the selected classification in map
Map = geemap.Map()
Map.add_basemap('HYBRID')
Map.centerObject(Region,6)
Map.addLayer(one.clip(Region),{'min':0,'max':1},'One')
Map

Map(center=[31.20907473717083, 120.33219382209627], controls=(WidgetControl(options=['position', 'transparent_…

##### Export the classified_img

In [131]:
Classification_instances_df

,classificaiton_tpye,year,classified_img
0,Spectrum_Normalize_Fourier_Terrain_Meterology,1990_1992,"ee.Image({\n ""functionInvocationValue"": {\n ..."
1,Spectrum_Normalize_Fourier_Terrain_Meterology,1993_1995,"ee.Image({\n ""functionInvocationValue"": {\n ..."
2,Spectrum_Normalize_Fourier_Terrain_Meterology,1996_1998,"ee.Image({\n ""functionInvocationValue"": {\n ..."
3,Spectrum_Normalize_Fourier_Terrain_Meterology,1999_2001,"ee.Image({\n ""functionInvocationValue"": {\n ..."
4,Spectrum_Normalize_Fourier_Terrain_Meterology,2002_2004,"ee.Image({\n ""functionInvocationValue"": {\n ..."
5,Spectrum_Normalize_Fourier_Terrain_Meterology,2005_2007,"ee.Image({\n ""functionInvocationValue"": {\n ..."
6,Spectrum_Normalize_Fourier_Terrain_Meterology,2008_2010,"ee.Image({\n ""functionInvocationValue"": {\n ..."
7,Spectrum_Normalize_Fourier_Terrain_Meterology,2011_2013,"ee.Image({\n ""functionInvocationValue"": {\n ..."
8,Spectrum_Normalize_Fourier_Terrain_Meterology,2014_2016,"ee.Image({\n ""functionInvocationValue"": {\n ..."
9,Spectrum_Normalize_Fourier_Terrain_Meterology,2017_2019,"ee.Image({\n ""functionInvocationValue"": {\n ..."


In [133]:
selecte_year = '2020_2022' # or None

if selecte_year:
    Classification_instances_df = Classification_instances_df >> mask(X.year==selecte_year)

    
for idx,item in Classification_instances_df.iterrows():
     
    Classification_type = item[0]
    year = item[1]
    img  = item[2].toInt8()
    
    
    export_name = f'{Classification_type}_China_{year}_patch'
    
    #export to GEE asset    
    task = ee.batch.Export.image.toAsset(   image          = img,
                                            description    = export_name,
                                            assetId        = f'{export_path}/{export_name}',
                                            region         = Region.geometry(),
                                            scale          = 30,
                                            maxPixels      = int(1e13)) 
    
    task.start()
    
    
    print(f'Exporting {export_name} successful!')

Exporting Spectrum_Normalize_Fourier_Terrain_Meterology_China_2020_2022_patch successful!
